# Proyecto Final Humanidades Digitales - FSalasNoguera

Este script hace peticiones al API de Spotify

1. ### Se importan las librerias y se pone el Token de Auth para el API de Spotify



In [1]:
#Importación de librerías
import datetime
import requests
import csv
import json
import random
import pandas as pd
import csv



# Timestamp como identificador de los archivos CSV
d = datetime.datetime.now()
timestamp = d.strftime("%Y-%m-%dT%H%M%S")



# Token de auth
token = 'BQDdfhXVIXtRllCUuh2bD2H21BBSvm7ePVOE-k-pMeeRXfemBOaVhjR2GjohvNDOb9mLQ4X2SNTJXjPYN27B6zf66T22gnRn9aHSJW-BLuBa2MqHjrYTIwbSKDaGUpkanD_feuQSpxYCTAnLcNHCCJCv1NAIBWc9Y6P35V5JKAgrJF9h80u-Rethms0Ecro'
username = "felipesalas1234"

# Para obtener el Auth toca usar oAuth2.0, esto lo estoy haciendo desde Postman



2. ### Ingresa por teclado el nombre de un artista y lo busca en Spotify 

In [2]:
#URL base del API

baseUrl = "https://api.spotify.com/v1/"

#Pide un input para buscar a un artista

artistNameInput = input()

id = 0


##https://api.spotify.com/v1/search?q=artistNameInput&type=artist&limit=1

url = "https://api.spotify.com/v1/" + "search?q=" + artistNameInput + "&type=artist&limit=1"



payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + token
}

response = requests.request("GET", url, headers=headers, data=payload)

#Si la respuesta del servidor es 200 o que fue satisfactoria almacena la respuesta JSON en una variable

if response.status_code == 200:

    searchArtist = response.json()

else: print(response,status_code)

#Obtiene el nombre del artista, su popularidad y ID
artists = searchArtist['artists']
theArtist = artists['items']
searchArtistName = theArtist[0]
theArtistName = searchArtistName['name']
theArtistId = searchArtistName['id']
theArtistPop = searchArtistName['popularity'] 

print("El artista ingresado es " + theArtistName)
print("Su popularidad es " + str(theArtistPop))
print("Su popularidad es " + str(theArtistId))

id = theArtistId

#Añade el nombre del artista en una variable para usar más adelante
artista = theArtistName


print("El artista ingresado es " + artista)

#Nombre del archivo de salida
csv_out = artista + "TracksAndRelated" + timestamp + ".csv"



#print(response.text)
    

florence and the machine
El artista ingresado es Florence + The Machine
Su popularidad es 78
Su popularidad es 1moxjboGR7GNWYIMWsRjgG
El artista ingresado es Florence + The Machine


### Opcional, buscar el artista por ID

In [ ]:

#Opcional, Obtener info del artista por ID

id = input()

url = "https://api.spotify.com/v1/" + "artists/" + id

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + token
}

response = requests.request("GET", url, headers=headers, data=payload)

if response.status_code == 200:

    artist = response.json()

else: print(response,status_code)

  # Store the artist name in the environment
artista = artist['name']


print("El artista ingresado es " + artista)



3. ### Obtenemos artistas relacionados del artista

In [3]:
#URL para hacer el request GET para obtener los artistas relacionados con el artista que se ingreso anteriormente
url = "https://api.spotify.com/v1/artists/" + id + "/related-artists"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + token
}

response = requests.request("GET", url, headers=headers, data=payload)


if response.status_code == 200:

    responseRelated = response.json()
    
else: print(response,status_code)

#contador para el for y Lista de los artistas relacionados
count = 0
idRelated = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for artistNum in responseRelated['artists']:
    artistsNames = artistNum['name']
    artistsIds = artistNum['id']
    #Imprime cada artista relacionado que informa Spotify
    print(artistsNames)
    idRelated [count] = artistNum['id']
    count += 1
    
    


            



Kate Nash
Lykke Li
La Roux
Regina Spektor
Paloma Faith
MARINA
Lily Allen
Robyn
Of Monsters and Men
HAIM
London Grammar
Bombay Bicycle Club
Janelle Monáe
The xx
Tegan and Sara
Lorde
Mark Ronson
BANKS
Daughter
Birdy


4. ### Se crea un playlist en la cuenta del usuario

In [4]:
#URL para el request POST para crear la playlist en el usuario

url = "https://api.spotify.com/v1/users/"+ username +"/playlists"

payload = json.dumps({
  "name": artista + " Mix",
  "public": False
})
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer '+token,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

responsePlaylist = response.json()

#Obtiene los datos del JSON y los almacena en las variables
elNombre = responsePlaylist['name']
playlistId = responsePlaylist['id']

print("El nombre de la playlist creada es: " + elNombre)



El nombre de la playlist creada es: Florence + The Machine Mix


5. ### Toma los Top Tracks del artista y los artistas relacionados. crea un CSV con los datos y los prepara para enviarlos a spotify





In [7]:
#Contador para el for de las canciones
count = 0

#Pepara el CSV para guardar los datos obtenidos de Spotify
with open(csv_out, 'w') as csvfile:

  
  #Instanciación del objeto writer de la librería CSV
  tablewriter = csv.writer(csvfile, delimiter=',', quotechar='"')
  

  # Headers para el CSV
  headers = ["track_title","track_id","track_artist","track_pop","track_uri","track_rDate"]

   # Se escriben los encabezados en la tabla
  tablewriter.writerow(headers)

  # campos a buscar en los metadatos
  track_title = ""
  track_id = ""
  track_artist = ""
  track_pop = ""
  track_uri = ""
  track_rDate = ""



  #Toma los top tacks de las bandas relacionadas
  #Contadores para el For y los IF
  countb = 0
  counta = 0
  #Lista de los tracks relacionados, crea 90 y los llena con un dato Dummie
  idTracks = ["spotify:track:3ZLElfsFXSMmNCliIGhIb7"] * 90

  #Para cada artista relacionado hace una petición para obtener sus top tracks
  for band in idRelated:
      url = "https://api.spotify.com/v1/artists/" + idRelated[countb] + "/top-tracks?country=co"
      payload={}
      headers = {
      'Accept': 'application/json',
      'Authorization': 'Bearer '+token,
      'Content-Type': 'application/json'
      }

      response = requests.request("GET", url, headers=headers, data=payload)

    
      
      if response.status_code == 200:

        responseTracks = response.json()
      
      else: print(response.status_code)

      
     #contador para el for
      countd = 0
      for trackId in responseTracks['tracks']:
        

        #Toma datos del Json
        track_title = trackId['name']
        print(track_title)
        track_id = trackId['id']
        countc = 0
        for trackArt in trackId['artists']:
          
          if countc > 0:
            track_artist = track_artist + " - " + trackArt['name']
          else:
            track_artist = trackArt['name']
          countc += 1
          print(track_artist)
        track_pop = trackId['popularity']
        track_uri = trackId['uri']
        track_rDate = trackId['album']['release_date']

        new_row = [track_title,track_id,track_artist,track_pop,track_uri,track_rDate]

        #escribimos la fila en la tabla
        tablewriter.writerow(new_row) 

        print(countd)

        #toma solamente tres tracks para añadirlo a la playlist creada los otros datos son solamente para el CSV
        if countd <= 3:
        
          #trackIds = trackId['id']
          idTracks [counta] = trackId['uri']
          countd += 1
          counta += 1
        else: print("Artista número: " + str(countb))
        

      countb += 1 

  print(idTracks)

  #Añade las canciones del artista principal

  url = "https://api.spotify.com/v1/artists/" + id + "/top-tracks?country=co"
  payload={}
  headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer '+token,
  'Content-Type': 'application/json'
  }

  response = requests.request("GET", url, headers=headers, data=payload)

    
      
  if response.status_code == 200:

      responseTracks = response.json()

  else: print(response.status_code)

      
  #Añade las canciones del artista principal a la lista de tracks  
  for trackId in responseTracks['tracks']:
      #trackIds = trackId['id']
      idTracks [counta] = trackId['uri']
      track_title = trackId['name']
      print(track_title)
      track_id = trackId['id']
      countc = 0
      for trackArt in trackId['artists']:
          
        if countc > 0:
          track_artist = track_artist + " - " + trackArt['name']
        else:
          track_artist = trackArt['name']
        countc += 1
        print(track_artist)
        track_pop = trackId['popularity']
        track_uri = trackId['uri']
        track_rDate = trackId['album']['release_date']

        new_row = [track_title,track_id,track_artist,track_pop,track_uri,track_rDate]

        #escribimos la fila en la tabla
        tablewriter.writerow(new_row) 
      counta += 1


Foundations
Kate Nash
0
Misery
Kate Nash
1
Merry Happy
Kate Nash
2
Nicest Thing
Kate Nash
3
Do-Wah-Doo
Kate Nash
4
Artista número: 0
Mouthwash
Kate Nash
4
Artista número: 0
Body Heat
Kate Nash
4
Artista número: 0
Life in Pink
Kate Nash
4
Artista número: 0
Dickhead
Kate Nash
4
Artista número: 0
Birds
Kate Nash
4
Artista número: 0
sex money feelings die - slowed version
Lykke Li
0
I Follow Rivers - The Magician Remix
Lykke Li
Lykke Li - The Magician
1
sex money feelings die
Lykke Li
2
Little Bit
Lykke Li
3
I Follow Rivers
Lykke Li
4
Artista número: 1
Late Night Feelings (feat. Lykke Li)
Mark Ronson
Mark Ronson - Lykke Li
4
Artista número: 1
No Rest for the Wicked
Lykke Li
4
Artista número: 1
I Know Places
Lykke Li
4
Artista número: 1
Unchained Melody
Lykke Li
4
Artista número: 1
Late Night Feelings - Jax Jones Midnight Snack Remix
Mark Ronson
Mark Ronson - Lykke Li
Mark Ronson - Lykke Li - Jax Jones
4
Artista número: 1
Bulletproof
La Roux
0
Bulletproof
La Roux
La Roux - GAMPER & DADONI
1

6. ### Desorganiza los items y los añade al playlist

In [6]:
#Random el orden las pistas de la playlist

random.shuffle(idTracks)

print(idTracks)

#Prepara los URIS para que Spotify lo acepte

uris = ",".join(idTracks)

print(uris)

url = "https://api.spotify.com/v1/users/felipesalas1234/playlists/" + playlistId + "/tracks?uris=" + uris

payload={}
headers = {'Accept': 'application/json','Authorization': 'Bearer '+token,'Content-Type': 'application/json'}

response = requests.request("POST", url, headers=headers, data=payload)


    
if response.status_code == 200:

    responseTracks = response.json()
else: print(response.status_code)

print("se ha creado la playlist en la cuenta de: " + username)

['spotify:track:59S3SiDxj914oyXd25zt3j', 'spotify:track:555CH6CnGJRqO37pqA30vW', 'spotify:track:0dkvbYmXYEPK7gGMdytmfI', 'spotify:track:1J2WmkS1Z18AuDWZCgIC5K', 'spotify:track:4yYvWFUWMaKMXtoJKJnmZk', 'spotify:track:0Ps4Eh6SQjeLnubbCsQTDD', 'spotify:track:2sEk5R8ErGIFxbZ7rX6S2S', 'spotify:track:4xEDawqLVwsC331aF6Z33x', 'spotify:track:6SluaPiV04KOaRTOIScoff', 'spotify:track:27mT3JdR3sRJyiMBFHdhB4', 'spotify:track:5gW5dSy3vXJxgzma4rQuzH', 'spotify:track:3q8iHOd2icirSipnkE7fae', 'spotify:track:56N9s9PS6VwGolUnx1CJkv', 'spotify:track:0g4fzRkbLeCDUCoe5iUOcf', 'spotify:track:4cLxUG9hsix8lj6woZ4j3w', 'spotify:track:5sF7hdYvhrcgNq8NZjfK8L', 'spotify:track:02dRkCEc8Q5ch4TTcnLxOn', 'spotify:track:4f5LDR4RWrboZT1YoSUfcy', 'spotify:track:5wD6de7Fz1QfxVNkv1SEy7', 'spotify:track:7p0tz5SeCrSxTLSvwR8W3D', 'spotify:track:2n0U2OG5d6TuW5mKx7YrC0', 'spotify:track:6Z0CSptkTKzOpownOPnK5g', 'spotify:track:3HNTLCpAGnizbyJmk7466r', 'spotify:track:5Zz8mrmVVhOq3iuv8Gh0MX', 'spotify:track:6k1I9gABiasGZiWWAosAEb',